# Projeto final

## Business question
O investidor James Bauer gostaria de diversificar seus negócios e começar a investir em imóveis. 
Ele definiu que compraria imóveis na cidade de Nova York, nos Estados Unidos. 
Por ser um dos locais mais caros para se viver no País, ele acredita que obterá um retorno satisfatório de seus investimentos caso loque imóveis na cidade. 

Como todas as suas decisões são tomadas com base em dados, ele contratou você, cientista de dados, para ajudá-lo nessa empreitada.
James Bauer planeja inicialmente locar os imóveis adquiridos e por isso ele definiu que irá utilizar a plataforma Airbnb para esse fim. Para isso, ele lhe entregou uma base de dados públicos da empresa, contendo os dados do comportamento dos hosts e de seus imóveis.

### O Desafio
James Bauer o contratou para realizar o estudo da base de dados fornecida e ajudá-lo com a escolha das regiões onde:
- há maior locação e maiores preços,
- fiquem em regiões favoráveis da cidade de Nova York, pois ele acredita que essas características irão ajudá-lo a recuperar o dinheiro investido na aquisição desses imóveis mais rapidamente. 

## Imports

In [57]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import gridspec
from matplotlib import pyplot as plt
import ipywidgets as widgets
import plotly.express as px
from ipywidgets import interact, interactive, fixed, interact_manual


## Auxiliary functions 

In [58]:
## # Supress Scientific Notation

np.set_printoptions(suppress=True)
pd.set_option('display.float_format', '{:.2f}'.format)

## Loading data

In [74]:
df=pd.read_csv('Datasets/AB_NYC_2019.csv')

## Dataset info

In [75]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.65,-73.97,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75,-73.98,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.81,-73.94,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.69,-73.96,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.80,-73.94,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

In [77]:
print( 'Number of houses available for purchase: {}'.format( df.shape[0] ) )
print( 'Features by house: {}'.format( df.shape[1] ) )

Number of houses available for purchase: 48895
Features by house: 16


In [83]:
len(df['id'].unique())

48895